# MQTT

## So What is MQTT?
    1999年由 Dr Andy Standford-Clark 和 Dr.Arlen Nipper 發明的通訊協議。當時開發的目的是為了可以在小頻寬和微小店裡損耗的情況下提供輕量，可靠的通訊協定。
    MQTT是一種基於「發布∕訂閱」機制的訊息傳輸協定
    代理人（broker）是個伺服器軟體，向伺服器發送主題的一方是發布者（publisher），從伺服器獲取主題的一方則是訂閱者（subscriber）
    「訂閱者」需要告知代理人想要訂閱的主題，每當「發布者」傳入新訊息時，代理人就會依照主題，傳送給所有訂閱者。
    「發布者」和「訂閱者」都是用戶端，代理人是伺服器。由於兩個用戶端之間有伺服器當作中繼站，所以兩邊並不需要知道彼此的IP位址。
    
    更多資訊可參訪https://swf.com.tw/?p=1002

測試環境
Mac OS 10.13.5
使用的是Eclipse Mosquitto 的broker https://mosquitto.org (支援Windows, Mac, Linux, Raspberry Pi, iPhone)
Library: eclipse/paho.mqtt.c https://github.com/eclipse/paho.mqtt.c
Compiler: GCC

Library function 請參考：
https://www.eclipse.org/paho/files/mqttdoc/MQTTClient/html/_m_q_t_t_client_8h.html#ab0d69ca03e618ccff0f175c3862d009b


1. 在下載的library 檔案輸入以下指令
    make
    sudo make install
2. 成功后應該會產生一個檔案build
3. 請把 build/output 裡的所有 .so 檔案複製到/usr/local/lib
4. 編譯請用 gcc -L/home/<USERNAME>/DIRECTORY TO DOWNLOAD LIBARY/build/output YOUR_FILE.c -lpaho-mqtt3c
    
    https://github.com/eclipse/paho.mqtt.cpp/issues/1 (Jason-Gew)
    https://stackoverflow.com/questions/32005906/example-mqtt-client-code-not-working-c

## Broker connection

In [ ]:
#include "MQTTClient.h"
#include <stdio.h>
#include <stdlib.h>

int main(){
    
    MQTTClient client;
	MQTTClient_connectOptions conn_opts = MQTTClient_connectOptions_initializer;
    
    /* Connection parameters */
	MQTTClient_create(&client, "tcp://localhost:1883", "A", MQTTCLIENT_PERSISTENCE_NONE, NULL); //"A" 是連線的名字
    
    int rc;
    int ch;
    
	/* Connect to MQTT Server */
    //MQTTClient_connect 會回傳一個值
	if ((rc = MQTTClient_connect(client, &conn_opts)) != MQTTCLIENT_SUCCESS){
		printf("Failed\n");
	} else{
		printf("Connected\n");
	}
    
    //Wait Until q or Q is entered
    do 
    {
        ch = getchar();
    } while(ch!='Q' && ch != 'q');
    
    
    //結束連線
    MQTTClient_disconnect(client, 10000);
    //清理client object
    MQTTClient_destroy(&client);
    return 0;
}

## Subscribe

In [ ]:
#include "MQTTClient.h"
#include <stdio.h>
#include <stdlib.h>

int msgarrvd(void *context, char *topicName, int topicLen, MQTTClient_message *message){
 	
 	int i;
    char* payloadptr;

    printf("Message arrived\n");
    printf("     topic: %s\n", topicName);
    printf("   message: ");

    payloadptr = message->payload;
    for(i=0; i<message->payloadlen; i++)
    {
        putchar(*payloadptr++);
    }
    putchar('\n');

    //需要把memory free 掉。Note 兩個free function 都需使用
    MQTTClient_freeMessage(&message);
    MQTTClient_free(topicName);
    return 1;
}

int main(){

    ...

	//設置觸發function 要在connect前設置
    //后三個parameter可設置callback分別為connectionlost, msgarrived, delivered
	MQTTClient_setCallbacks(client, NULL, NULL, msgarrvd, NULL);

	int rc;
	int ch;

	/* Connect to MQTT Server */
	if ((rc = MQTTClient_connect(client, &conn_opts)) != MQTTCLIENT_SUCCESS){
		printf("Failed\n");
	} else{
		printf("Connected\n");
	}

	MQTTClient_subscribe(client, "/example", 1);
	printf("Successfully Subscribed to the topic\n");

	 do 
    {
        ch = getchar();
    } while(ch!='Q' && ch != 'q');

    //結束連線
    MQTTClient_disconnect(client, 10000);
    //清理client object
    MQTTClient_destroy(&client);
	return rc;
}

## Publish

In [ ]:
#include "MQTTClient.h"
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main(){

	MQTTClient client;
	MQTTClient_connectOptions conn_opts = MQTTClient_connectOptions_initializer;
	MQTTClient_message pubmsg = MQTTClient_message_initializer;

    /* 把message放進pubmsg*/
	pubmsg.payload = "Hello a";
	pubmsg.payloadlen = strlen("Hello a");
	pubmsg.qos = 1;
	pubmsg.retained = 0;


	/* Connection parameters */
	MQTTClient_create(&client, "tcp://localhost:1883", "Ab", MQTTCLIENT_PERSISTENCE_NONE, NULL);
	int rc;
    
    /* Optional */
	conn_opts.keepAliveInterval = 20;
	conn_opts.cleansession = 1; 

	/* Connect to MQTT Server */
	if ((rc = MQTTClient_connect(client, &conn_opts)) != MQTTCLIENT_SUCCESS){
		printf("Failed\n");
	} else{
		printf("Connected\n");
	}
    
    /* 有兩種publish message 的 method 兩種都可以使用*/
    /* 1.*/
	MQTTClient_publish(client,"/example",pubmsg.payloadlen,pubmsg.payload,pubmsg.qos,pubmsg.retained,NULL);
    printf("Message 1 published\n");
    
    /* 2.*/
    MQTTClient_publishMessage(client,"/example", &pubmsg,NULL);
    
	printf("Message 2 published\n");

	return 0; 
}

## Async Publish

In [ ]:
#include "MQTTClient.h"
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
volatile MQTTClient_deliveryToken deliveredtoken;


int flag = 1; 
void connlost(void *context, char *cause){
	printf("Connection Lost\n");
	printf("Reason: %s\n",cause);
}

/* Note this function will only be called when QOS is 1 */
void delivered(void *context, MQTTClient_deliveryToken dt){
	printf("Message Successfully Delivered\n");
	printf("The token is %d\n", dt);
	flag = 0; 
	deliveredtoken = dt; 
}

int msgarrvd(void *context, char *topicName, int topicLen, MQTTClient_message *message)
{
    int i;
    char* payloadptr;

    printf("Message arrived\n");
    printf("     topic: %s\n", topicName);
    printf("   message: ");

    payloadptr = message->payload;
    for(i=0; i<message->payloadlen; i++)
    {
        putchar(*payloadptr++);
    }
    putchar('\n');
    MQTTClient_freeMessage(&message);
    MQTTClient_free(topicName);
    return 1;
  }

int main(int argc, char const *argv[])
{
	...
	MQTTClient_deliveryToken token;
	deliveredtoken = 0 ; 
    
    //設置觸發function 要在connect前設置
    /*
    *  這裡有個bug 這個function 要求msgarrvd 一定要存在否則return error 
	*  msgarrvd 提供后delivered 才能使用
    */
	MQTTClient_setCallbacks(client, NULL, connlost, msgarrvd, delivered);
    
    ...

	while(deliveredtoken != token)
	//while(flag != 0)	
		;

	//結束連線
   	MQTTClient_disconnect(client, 10000);
   	//清理client object
    MQTTClient_destroy(&client);
	
	return 0;
}